We load and preprocess the sdp file

In [4]:
#needed dependencies for io in Drive
!pip install -U -q PyDrive
!pip install h5py
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
from google.colab import drive
drive.mount('/content/drive')
!ls "/content/drive/MyDrive/CST/L98/"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
'Biaffine Parser.ipynb'   indexed_data.py  'Output Full Model.gdoc'
 biaffine_parser.py	  indexing.py	    readme
 data.py		  l1-rerank.sdp    'Training Biaffine Semantic Dependency Parser.ipynb'
 dm.sdp			  l2-rerank.sdp     train_or_use_parser.py
 eval_dep.py		  model
'Evaluation Code.gdoc'	  modules.py


In [5]:
f = open("/content/drive/MyDrive/CST/L98/dm.sdp")
sentences = {}
current = ["", ""]
col_names = [["sentid"], ["form"], ["lemma"]]
index = []
for line in f.readlines():
  if len(line) > 0:
      if line[0] == "#":
          if current[0] != "#":
              sentences[current[0]] = current[1]
              index.append(current[0])
          current[0] = int(line[1:])
          current[1] = ""
      else:
          current[1] += line

#sentences[current[0]] = current[1]

print(sentences[20001001])

1	Pierre	Pierre	NNP	-	+	_	_	_	_	_	_	_	_	_	_	_
2	Vinken	_generic_proper_ne_	NNP	-	-	compound	_	_	ARG1	ARG1	_	_	_	_	_	_
3	,	_	,	-	-	_	_	_	_	_	_	_	_	_	_	_
4	61	_generic_card_ne_	CD	-	+	_	_	_	_	_	_	_	_	_	_	_
5	years	year	NNS	-	+	_	ARG1	_	_	_	_	_	_	_	_	_
6	old	old	JJ	-	+	_	_	measure	_	_	_	_	_	_	_	_
7	,	_	,	-	-	_	_	_	_	_	_	_	_	_	_	_
8	will	will	MD	-	-	_	_	_	_	_	_	_	_	_	_	_
9	join	join	VB	+	+	_	_	_	_	_	_	ARG1	_	_	_	_
10	the	the	DT	-	+	_	_	_	_	_	_	_	_	_	_	_
11	board	board	NN	-	-	_	_	_	_	ARG2	BV	_	_	_	_	_
12	as	as	IN	-	+	_	_	_	_	_	_	_	_	_	_	loc
13	a	a	DT	-	+	_	_	_	_	_	_	_	_	_	_	_
14	nonexecutive	_generic_jj_	JJ	-	+	_	_	_	_	_	_	_	_	_	_	_
15	director	director	NN	-	-	_	_	_	_	_	_	ARG2	BV	ARG1	_	_
16	Nov.	Nov.	NNP	-	+	_	_	_	_	_	_	_	_	_	_	_
17	29	_generic_dom_card_ne_	CD	-	+	_	_	_	_	_	_	_	_	_	of	_
18	.	_	.	-	-	_	_	_	_	_	_	_	_	_	_	_




We train the model on 30K+ sentences using a T4 GPU

In [7]:
!pip install sacremoses
!python3 "/content/drive/MyDrive/CST/L98/train_or_use_parser.py" train "/content/drive/MyDrive/CST/L98/dm.sdp" "/content/drive/MyDrive/CST/L98/model/" -g sdp -v "/content/drive/MyDrive/CST/L98/dm.sdp" -e loss

2023-12-06 15:15:12.494992: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-06 15:15:12.495055: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-06 15:15:12.495104: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-06 15:15:13.970930: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
tokenizer_config.json: 100% 29.0/29.0 [00:00<00:00, 185kB/s]
config.json: 100% 570/570 [00:00<00:00, 3.72MB/s]
vocab.txt: 100% 213k/213k [00:00<00:00, 1.09MB/s]
tokenizer.json: 100% 436k/436k [00:00<00:00, 1.52MB/s]
There are 1 GPU(s) ava

We test the model on l1-rerank

In [11]:
!python3 "/content/drive/MyDrive/CST/L98/train_or_use_parser.py" test "/content/drive/MyDrive/CST/L98/l1-rerank.sdp" "/content/drive/MyDrive/CST/L98/model/" -g sdp --out_parsed_file training.txt

2023-12-06 16:27:51.685705: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-06 16:27:51.685772: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-06 16:27:51.685814: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-06 16:27:53.046322: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Parsing /content/drive/MyDrive/CST/L98/l1-rerank.sdp with /content/drive/MyDrive/CST/L98/model//model ...
loaded model /content/drive/MyDrive/CST/L98/model//model
Max sentence length: {'toparse': 63}
parsing and evaluating conll file /con

In [13]:
!python3 "/content/drive/MyDrive/CST/L98/eval_dep.py" training.txt.l

ALL UR 82.47 UP 81.14 UF 81.80
ALL LR 76.60 LP 75.36 LF 75.98
>40 UR 70.73 UP 82.17 UF 76.02
>40 LR 66.62 LP 77.39 LF 71.60
LABEL          ARG1 GOLD DIST:  0.78 (MAD  2.47) PRED DIST:  0.46 (MAD  2.43)
LABEL          ARG1 GOLD DIR :  0.24 (MAD  0.94) PRED DIR :  0.19 (MAD  0.97)
LABEL          ARG2 GOLD DIST: -1.79 (MAD  1.25) PRED DIST: -1.84 (MAD  1.25)
LABEL          ARG2 GOLD DIR : -0.85 (MAD  0.27) PRED DIR : -0.84 (MAD  0.29)
LABEL          ARG3 GOLD DIST: -2.75 (MAD  1.77) PRED DIST: -2.35 (MAD  1.24)
LABEL          ARG3 GOLD DIR : -0.90 (MAD  0.19) PRED DIR : -0.97 (MAD  0.06)
LABEL          ARG4 GOLD DIST: -1.50 (MAD  3.50) PRED DIST:  0.00 (MAD  0.00)
LABEL          ARG4 GOLD DIR : -0.73 (MAD  0.47) PRED DIR :  0.00 (MAD  0.00)
LABEL            BV GOLD DIST: -1.39 (MAD  0.53) PRED DIST: -1.40 (MAD  0.55)
LABEL            BV GOLD DIR : -1.00 (MAD  0.00) PRED DIR : -1.00 (MAD  0.00)
LABEL            BV : proportion of odd direction in predicted arcs :  0.07 (4 / 5426)
LABEL   _

l2-rerank

In [14]:
!python3 "/content/drive/MyDrive/CST/L98/train_or_use_parser.py" test "/content/drive/MyDrive/CST/L98/l2-rerank.sdp" "/content/drive/MyDrive/CST/L98/model/" -g sdp --out_parsed_file l2.txt

2023-12-06 16:37:11.038746: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-06 16:37:11.038811: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-06 16:37:11.038850: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-06 16:37:12.854876: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Parsing /content/drive/MyDrive/CST/L98/l2-rerank.sdp with /content/drive/MyDrive/CST/L98/model//model ...
loaded model /content/drive/MyDrive/CST/L98/model//model
Max sentence length: {'toparse': 59}
parsing and evaluating conll file /con

In [15]:
!python3 "/content/drive/MyDrive/CST/L98/eval_dep.py" l2.txt.l

ALL UR 78.17 UP 77.46 UF 77.81
ALL LR 72.43 LP 71.78 LF 72.10
>40 UR 63.75 UP 76.92 UF 69.71
>40 LR 60.00 LP 72.40 LF 65.62
LABEL          ARG1 GOLD DIST:  0.85 (MAD  2.41) PRED DIST:  0.53 (MAD  2.33)
LABEL          ARG1 GOLD DIR :  0.26 (MAD  0.93) PRED DIR :  0.21 (MAD  0.96)
LABEL          ARG2 GOLD DIST: -1.68 (MAD  1.44) PRED DIST: -1.80 (MAD  1.23)
LABEL          ARG2 GOLD DIR : -0.83 (MAD  0.32) PRED DIR : -0.85 (MAD  0.28)
LABEL          ARG3 GOLD DIST: -2.44 (MAD  2.29) PRED DIST: -2.26 (MAD  1.25)
LABEL          ARG3 GOLD DIR : -0.80 (MAD  0.37) PRED DIR : -0.97 (MAD  0.07)
LABEL          ARG4 GOLD DIST: -2.38 (MAD  2.79) PRED DIST:  0.00 (MAD  0.00)
LABEL          ARG4 GOLD DIR : -0.69 (MAD  0.52) PRED DIR :  0.00 (MAD  0.00)
LABEL            BV GOLD DIST: -1.39 (MAD  0.53) PRED DIST: -1.39 (MAD  0.54)
LABEL            BV GOLD DIR : -1.00 (MAD  0.00) PRED DIR : -1.00 (MAD  0.00)
LABEL            BV : proportion of odd direction in predicted arcs :  0.09 (3 / 3515)
LABEL    